In [1]:
import pandas as pd
import numpy as np
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.compose import make_column_transformer

решил взять этот датасет https://www.kaggle.com/colearninglounge/predicting-pulsar-starintermediate
В нем собраны данные о звездах и предстоит предсказать превратиться ли звезда в пульсар или нет. Поскольку пульсары достаточно редкие, то данные существенно разрежены.
При этом сами данные прдеставляют собой статистические оценки распределения электромагнитных импульсов нейтронных звезд.
Первые четыре фичи - это оценки приходящего на змелю импульса от конкртеного объекта, причем насколько я понял испульсы от конкретно пульсаров приходят не с одной частотой и имеют разную форму и могут иметь нехарактерные выбросы. Поэтому астрономы собирают по конкретной звезде сигнатуры импульсов и усредняют их и на выходе получаются 4 первые фичи которые есть в датасете.  
  
А остальные 4 - оценки DM-SNR curve....насколько я понял это связано тоже с импульсом, только с тем, что импульс приходит с  шумом и на разных частотах, связанно это с ионизаией межзвездного пространства и называется в англоязычных источниках "dispersion" - не знаю как наши астрономы это называют, но переводить как дисперсию не совсем корректно как мне кажется. В общем каким-то образом астрономы считают для каждого приходящего сигнала от звезды "dispersion-measure-signal-to-noise-ratio" и так же их усредняют. Вот собственно последние 4 фичи как раз и есть статистические оценки этих кривых для каждой звезды.

In [2]:
df = pd.read_csv("./data/HW_6/pulsar_data_train.csv")

In [3]:
df.dtypes

 Mean of the integrated profile                  float64
 Standard deviation of the integrated profile    float64
 Excess kurtosis of the integrated profile       float64
 Skewness of the integrated profile              float64
 Mean of the DM-SNR curve                        float64
 Standard deviation of the DM-SNR curve          float64
 Excess kurtosis of the DM-SNR curve             float64
 Skewness of the DM-SNR curve                    float64
target_class                                     float64
dtype: object

In [4]:
df.head(3)

,Mean of the integrated profile,Standard deviation of the integrated profile,Excess kurtosis of the integrated profile,Skewness of the integrated profile,Mean of the DM-SNR curve,Standard deviation of the DM-SNR curve,Excess kurtosis of the DM-SNR curve,Skewness of the DM-SNR curve,target_class
0,121.156250,48.372971,0.375485,-0.013165,3.168896,18.399367,7.449874,65.159298,0.0
1,76.968750,36.175557,0.712898,3.388719,2.399666,17.570997,9.414652,102.722975,0.0
2,130.585938,53.229534,0.133408,-0.297242,2.743311,22.362553,8.508364,74.031324,0.0


In [5]:
df.shape

(12528, 9)

In [6]:
#  примеров много, это хорошо. Посмотрим на баланс классов

In [7]:
df["target_class"].value_counts()

0.0    11375
1.0     1153
Name: target_class, dtype: int64

In [8]:
#  Видим существенный дисбаланс, посчти в 10 раз. Попробуем решить задачу PU-learning, но сначала попробуем пофитить данные.
#  Честно признаться что делать с такими данными ума не приложу. Но раз они все непрервные и иих мало, то предлагаю 
#  Для начала отмасштабировать их, а затем к каждой применить преобразование в **2 и логарифмическое преобразование

In [9]:
#  посмотрим сколько пропущенных значений

In [10]:
df.isna().sum()

 Mean of the integrated profile                     0
 Standard deviation of the integrated profile       0
 Excess kurtosis of the integrated profile       1735
 Skewness of the integrated profile                 0
 Mean of the DM-SNR curve                           0
 Standard deviation of the DM-SNR curve          1178
 Excess kurtosis of the DM-SNR curve                0
 Skewness of the DM-SNR curve                     625
target_class                                        0
dtype: int64

In [11]:
#  ну в целом встречаются, но их немного....сделаем пайплайн, который будет заменять пропущенные значения на медиану

In [12]:
from sklearn.model_selection import train_test_split

In [13]:
#  разделим данные
X_train, X_test, y_train, y_test = train_test_split(df.loc[:, :" Skewness of the DM-SNR curve"], df[["target_class"]], 
                                                    test_size=0.3, random_state=42)

In [14]:
class ReplaceNan(BaseEstimator, TransformerMixin):
    def __init__(self):
        self.median_dict = {}

    def fit(self, X, y=None):
        for col in X.columns:
            self.median_dict[col] = X[col].median()
        return self

    def transform(self, X):
        out = X.copy()
        for col in X.columns:
            out.loc[out[col].isna(), col] = self.median_dict[col]
        return out

In [15]:
class CreateDf(BaseEstimator, TransformerMixin):
    def __init__(self, col_names):
        self.col_names = col_names

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        out = X.copy()
        return pd.DataFrame(out, columns=self.col_names)

In [16]:
class PolyFeatureAll(BaseEstimator, TransformerMixin):

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        out = X.copy()
        for col in out.columns:
            out[f"{col}_squared"] = out[col] ** 2
            out[f"{col}_log"] = np.where(out[f"{col}_squared"] == 0, 
                                         0, 
                                         np.log(out[f"{col}_squared"])
                                        )
        return out

#  Поскольку я собираюьс использовать StandartScaler, а там могут присутсвовать отрицательные значения, что для логарифма 
#  неприемлимо, то в качестве аргумента логарифма я буду передавать квадрат соответсвующей фичи
#  полюс учесть вариант с нулем, потому что логарифм нуля не определен

In [17]:
scaler = make_column_transformer(
                                 (StandardScaler(), 
                                  [' Mean of the integrated profile', ' Standard deviation of the integrated profile',
                                   ' Excess kurtosis of the integrated profile', ' Skewness of the integrated profile', 
                                   ' Mean of the DM-SNR curve', ' Standard deviation of the DM-SNR curve', 
                                   ' Excess kurtosis of the DM-SNR curve', ' Skewness of the DM-SNR curve'])
                                 )

In [18]:
feature_engine = make_pipeline(scaler, 
                               CreateDf(["mean_of_intprof", "std_of_intprof", "kurtosis_of_intprof", "skewness_of_intprof",
                                         "mean_of_dmsnr", "std_of_dmsnr", "kurtosis_of_dmsnr", "skewness_of_dmsnr"]),
                               ReplaceNan())

In [19]:
feature_engine.fit_transform(X_train).describe()

,mean_of_intprof,std_of_intprof,kurtosis_of_intprof,skewness_of_intprof,mean_of_dmsnr,std_of_dmsnr,kurtosis_of_dmsnr,skewness_of_dmsnr
count,8.769000e+03,8.769000e+03,8769.000000,8.769000e+03,8.769000e+03,8769.000000,8.769000e+03,8769.000000
mean,1.917474e-16,-1.733513e-16,-0.033240,-9.422779e-18,3.089228e-18,-0.037475,-2.269063e-16,-0.010248
std,1.000057e+00,1.000057e+00,0.931692,1.000057e+00,1.000057e+00,0.959965,1.000057e+00,0.975907
min,-4.093953e+00,-3.146608e+00,-2.069390,-5.732479e-01,-4.209666e-01,-0.975739,-2.466672e+00,-1.003284
25%,-4.006069e-01,-6.092257e-01,-0.395392,-3.161286e-01,-3.634431e-01,-0.595377,-5.584327e-01,-0.631312
50%,1.573829e-01,6.148837e-02,-0.239312,-2.549724e-01,-3.334553e-01,-0.405699,2.869776e-02,-0.206107
75%,6.301579e-01,6.585041e-01,-0.054473,-1.362838e-01,-2.435483e-01,0.024536,5.326459e-01,0.288401
max,3.045283e+00,6.590582e+00,7.099863,1.066318e+01,6.626952e+00,4.232422,5.827287e+00,10.239105


In [20]:
feature_engine.fit_transform(X_train).isna().sum()

mean_of_intprof        0
std_of_intprof         0
kurtosis_of_intprof    0
skewness_of_intprof    0
mean_of_dmsnr          0
std_of_dmsnr           0
kurtosis_of_dmsnr      0
skewness_of_dmsnr      0
dtype: int64

In [21]:
# посмотрим как отработала замена пустых значений

In [22]:
df.loc[df[" Standard deviation of the DM-SNR curve"].isna()].head(5)
#  пропущенные значения по 6 фиче

,Mean of the integrated profile,Standard deviation of the integrated profile,Excess kurtosis of the integrated profile,Skewness of the integrated profile,Mean of the DM-SNR curve,Standard deviation of the DM-SNR curve,Excess kurtosis of the DM-SNR curve,Skewness of the DM-SNR curve,target_class
3,156.398438,48.865942,-0.215989,-0.171294,17.471572,NaN,2.958066,7.197842,0.0
5,121.007812,47.176944,0.229708,0.091336,2.036789,NaN,9.546051,112.131721,0.0
6,79.343750,42.402174,1.063413,2.244377,141.641304,NaN,-0.700809,-1.200653,0.0
9,109.156250,47.002234,0.394182,0.190296,4.578595,NaN,5.702532,36.342493,0.0
23,135.132812,47.871797,-0.155981,0.375567,5.229933,NaN,5.787467,39.372558,0.0


In [23]:
feature_engine.fit_transform(X_train).iloc[[3, 5, 6, 9, 23], :]

,mean_of_intprof,std_of_intprof,kurtosis_of_intprof,skewness_of_intprof,mean_of_dmsnr,std_of_dmsnr,kurtosis_of_dmsnr,skewness_of_dmsnr
3,0.482473,0.421053,-0.417937,-0.329279,-0.289601,-0.121914,-0.277934,-0.475524
5,0.446992,-0.178564,-0.288468,-0.302793,-0.329200,-0.380091,0.159656,-0.206107
6,0.823938,0.900998,-0.319447,-0.306827,-0.229484,0.163982,-0.646372,-0.706839
9,0.884286,-0.051642,-0.492155,-0.290261,-0.361527,-0.730961,0.642057,0.711932
23,0.798161,0.459699,-0.239312,-0.327363,3.608050,3.597527,-1.922417,-1.001159


In [24]:
#  вроде работает, посмотрим на всякий случай по другой фиче

In [25]:
df.loc[df[" Excess kurtosis of the integrated profile"].isna()].head(5)
#  пропущенные значения по 3 фиче

,Mean of the integrated profile,Standard deviation of the integrated profile,Excess kurtosis of the integrated profile,Skewness of the integrated profile,Mean of the DM-SNR curve,Standard deviation of the DM-SNR curve,Excess kurtosis of the DM-SNR curve,Skewness of the DM-SNR curve,target_class
13,128.734375,48.695560,NaN,0.307803,5.307692,25.162295,5.497544,32.739833,0.0
18,115.781250,53.305713,NaN,-0.518847,3.565217,22.372342,7.740922,65.024844,0.0
37,107.703125,44.726313,NaN,0.390775,2.598662,17.145047,8.498741,85.013918,0.0
51,107.070312,51.695198,NaN,-0.400658,1.832776,17.559706,10.530075,117.131775,0.0
55,120.593750,55.713895,NaN,-0.232142,3.531773,20.847988,8.088508,75.919711,0.0


In [26]:
feature_engine.fit_transform(X_train).iloc[[13, 18, 37, 51, 55], :]

,mean_of_intprof,std_of_intprof,kurtosis_of_intprof,skewness_of_intprof,mean_of_dmsnr,std_of_dmsnr,kurtosis_of_dmsnr,skewness_of_dmsnr
13,0.081569,-0.730737,-0.083880,-0.184643,-0.344588,-0.401857,0.131882,-0.147778
18,0.740240,0.448341,-0.239312,-0.392323,-0.291912,0.031074,-0.230584,-0.478105
37,-0.164068,0.250609,0.011857,-0.243380,-0.355354,-0.651438,0.359339,0.246921
51,0.582850,-0.027685,-0.277515,-0.214913,-0.360343,-0.715404,0.521740,0.561410
55,0.927348,0.540544,-0.533423,-0.338612,1.524547,2.833789,-1.648768,-0.994194


In [27]:
#  отлично, все отрабатывает вроде всерно
#  перезапишем трансформер добавив степенное преобразование

In [28]:
feature_engine = make_pipeline(scaler, 
                               CreateDf(["mean_of_intprof", "std_of_intprof", "kurtosis_of_intprof", "skewness_of_intprof",
                                         "mean_of_dmsnr", "std_of_dmsnr", "kurtosis_of_dmsnr", "skewness_of_dmsnr"]),
                               ReplaceNan(),
                               PolyFeatureAll()
                              )

In [29]:
feature_engine.fit_transform(X_train)

,mean_of_intprof,std_of_intprof,kurtosis_of_intprof,skewness_of_intprof,mean_of_dmsnr,std_of_dmsnr,kurtosis_of_dmsnr,skewness_of_dmsnr,mean_of_intprof_squared,mean_of_intprof_log,...,skewness_of_intprof_squared,skewness_of_intprof_log,mean_of_dmsnr_squared,mean_of_dmsnr_log,std_of_dmsnr_squared,std_of_dmsnr_log,kurtosis_of_dmsnr_squared,kurtosis_of_dmsnr_log,skewness_of_dmsnr_squared,skewness_of_dmsnr_log
0,0.181037,0.218106,-0.294831,-0.320942,-0.263841,0.044253,-0.426087,-0.582606,0.032774,-3.418110,...,0.103004,-2.272987,0.069612,-2.664819,0.001958,-6.235653,0.181550,-1.706223,0.339430,-1.080488
1,-0.080673,0.674244,-0.238607,-0.276213,-0.358370,-0.593524,0.434472,0.239128,0.006508,-5.034715,...,0.076293,-2.573169,0.128429,-2.052379,0.352271,-1.043356,0.188766,-1.667249,0.057182,-2.861514
2,0.961009,-0.219836,-0.479055,-0.206964,-0.251355,0.132838,-0.428847,-0.591623,0.923539,-0.079542,...,0.042834,-3.150418,0.063179,-2.761776,0.017646,-4.037255,0.183910,-1.693311,0.350018,-1.049770
3,0.482473,0.421053,-0.417937,-0.329279,-0.289601,-0.121914,-0.277934,-0.475524,0.232780,-1.457662,...,0.108425,-2.221701,0.083869,-2.478502,0.014863,-4.208875,0.077247,-2.560744,0.226123,-1.486675
4,1.004375,-0.721643,-0.418720,-0.185203,-0.375168,-0.604979,0.822467,0.512726,1.008769,0.008731,...,0.034300,-3.372603,0.140751,-1.960765,0.366000,-1.005122,0.676452,-0.390893,0.262888,-1.336027
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8764,-0.211982,-2.030838,-0.012646,0.205623,-0.347604,-0.342031,0.174245,-0.158935,0.044936,-3.102507,...,0.042281,-3.163424,0.120828,-2.113385,0.116985,-2.145706,0.030361,-3.494587,0.025260,-3.678520
8765,-0.071878,-0.302066,-0.037593,-0.173809,-0.403042,-0.405699,2.104268,2.279705,0.005166,-5.265567,...,0.030209,-3.499602,0.162443,-1.817431,0.164592,-1.804288,4.427942,1.487935,5.197055,1.648092
8766,-3.896534,-1.677625,4.761961,4.792705,2.076080,1.492015,-1.613207,-0.975166,15.182978,2.720175,...,22.970018,3.134190,4.310109,1.460963,2.226110,0.800256,2.602438,0.956449,0.950949,-0.050295
8767,-0.213498,0.222297,-0.239312,-0.220054,-0.308202,-0.283934,-0.229892,-0.413254,0.045582,-3.088252,...,0.048424,-3.027761,0.094989,-2.353997,0.080619,-2.518026,0.052850,-2.940293,0.170779,-1.767388


In [30]:
#  ради интереса обучу быстренько модель на решающих деревьях чтобы посмотреть насколько хорошо классический подход работает

In [31]:
from sklearn.ensemble import RandomForestClassifier

In [32]:
forest_clf = RandomForestClassifier(n_estimators=500, n_jobs=-1, max_depth=7, criterion="entropy")

In [33]:
forest_pipe = make_pipeline(feature_engine, forest_clf)

In [34]:
forest_pipe.fit(X_train, y_train.to_numpy().reshape(-1))

Pipeline(steps=[('pipeline',
                 Pipeline(steps=[('columntransformer',
                                  ColumnTransformer(transformers=[('standardscaler',
                                                                   StandardScaler(),
                                                                   [' Mean of '
                                                                    'the '
                                                                    'integrated '
                                                                    'profile',
                                                                    ' Standard '
                                                                    'deviation '
                                                                    'of the '
                                                                    'integrated '
                                                                    'profile',
                             

In [35]:
forest_clf_pred = forest_pipe.predict_proba(X_test)[:, 1]
forest_clf_pred

array([0.00572015, 0.00195588, 0.00314829, ..., 0.00807539, 0.01994045,
       0.37972618])

In [36]:
from sklearn.metrics import roc_auc_score

In [37]:
roc_auc_score(y_test, forest_clf_pred)

0.9797100267982347

In [38]:
#  хм.....а судя по всему получился неплохой классификатор....но надо посмотреть на confusion matrix... возможно 
#  из-за дисбаланса классов у нас такая хорошая оценка выходит

In [39]:
from sklearn.metrics import precision_recall_curve, confusion_matrix

In [41]:
precision, recall, thresholds = precision_recall_curve(y_test, forest_clf_pred)

fscore = (2 * precision * recall) / (precision + recall)
# locate the index of the largest f score
ix = np.argmax(fscore)
print('Best Threshold=%f, F-Score=%.3f, Precision=%.3f, Recall=%.3f' % (thresholds[ix], 
                                                                        fscore[ix],
                                                                        precision[ix],
                                                                        recall[ix]))

Best Threshold=0.395163, F-Score=0.894, Precision=0.949, Recall=0.844


In [44]:
confusion_matrix(y_test, forest_clf_pred>thresholds[ix])

array([[3390,   16],
       [  56,  297]], dtype=int64)

In [45]:
#  ну в целом действительно неплохой классификатор получился. Мы с уверенностью можем вычленить все пульсары из общего массива
#  при этом мы очень мало делаем ошибок второго рода, т.е. FP. Думаю для целей разведочного научного исследования это лучше,
#  поскольку не нужно тратить ресурсы на то чтобы вычленить FP примеры из предсказанного массива....как если бы precision Был
#  бы на уровне 50-60%

In [46]:
#  продолжение следуюет...